In [ ]:
#Necessary Imports
# !pip install ta
import numpy as np
import pandas as pd
import os
import pickle
import torch
# import ta
from google.colab import drive
drive.mount('/content/drive')

#Data Source Location
os.chdir("/content/drive/My Drive/mining-on-stock")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Load Feature Engineered Stock dataset: 100 Features including labels
with open('data_with_features_clean_train.pkl', 'rb') as train_file: #To do: Change file name.
  train_data = pickle.load(train_file)

In [ ]:
#Last 5 data points of the first stock
print(f'Last 5 datapoints of the first stock among 2000 stocks')
stock_len = 2201
train_data.iloc[stock_len-5: stock_len] #To do: Once the concatenated file is ready.

Last 5 datapoints of the first stock among 2000 stocks


,Open,High,Low,Close,Volume,awesome_oscillator (AO),bollinger_percent_b (percent_b),cfo (cfo),cmo (cmo),detrended_price_oscillator (dpo),...,ichimoku_clouds (senkou_span_a),ichimoku_clouds (chikou_span),keltner_channel (Keltner Center),keltner_channel (Keltner Upper),keltner_channel (Keltner Lower),linear_reg_forecast (LRF),mass_index (Mass_Index),median_price (Median Price),money_flow_index (Money Flow Index),Label
2197,0.680849,0.683633,0.672997,0.674055,0.351048,0.010157,0.744441,0.068595,21.168686,0.004813,...,0.683299,0.689504,0.677062,0.692473,0.661650,0.689504,1.080827,0.678315,7.843752,decrease
2198,0.674389,0.674890,0.671326,0.674055,0.143069,0.010020,0.723160,0.068595,19.300231,0.005677,...,0.682909,0.689504,0.677206,0.692105,0.662307,0.689504,0.966600,0.673108,7.179743,no big change
2199,0.674110,0.676227,0.672440,0.676004,0.312048,0.010509,0.755154,0.068595,21.857938,0.008792,...,0.682505,0.689504,0.677414,0.691979,0.662849,0.689504,0.896525,0.674333,7.179743,increase
2200,0.675725,0.675837,0.668820,0.669321,0.279230,0.008306,0.536443,0.068595,18.491048,0.003247,...,0.682505,0.689504,0.677475,0.692413,0.662537,0.689504,0.925453,0.672328,7.535366,decrease
2201,0.669293,0.671326,0.662973,0.664198,0.552933,0.005024,0.362143,0.068595,15.904361,-0.000906,...,0.679735,0.689504,0.677201,0.691448,0.662953,0.689504,0.977753,0.667149,7.707911,decrease


In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset
import multiprocessing

class MyDataset(Dataset):
    def __init__(self, df, stock_len, seq_length):
        self.X = []
        self.y = []
        
        labels = df.iloc[:, -1].values # Extract labels from the last column
        cnt = len(labels)
        
        df = df.iloc[:, :-1] # Remove labels column from the DataFrame        
        values = torch.tensor(df.values) # Convert DataFrame to tensor
        
        # Create input/output sequences
        while cnt != 0:
          for i in range(seq_length, stock_len):
            self.X.append(values[i-seq_length:i, :])
            self.y.append(labels[i])
          cnt -= stock_len
        
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    
    def __len__(self):
        return len(self.X)

def data_generator(train_data, stock_len, seq_length, batch_size, num_workers):
    train_dataset = MyDataset(train_data, stock_len, seq_length)
    
    # Create DataLoader
    # Optimzation 1: Dataset and Dataloaders for batch processing
    # Optimization 2: (num_of_workers, pin_memory(GPU specific))    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)

    for X_batch, y_batch in train_loader:
        # Convert labels to tensor
        y_train = np.array(y_batch)
        labels_encoded = torch.zeros((len(y_train), 3), dtype=torch.float32)
        labels_encoded[y_train == 'increase', 2] = 1
        labels_encoded[y_train == 'no big change', 1] = 1
        labels_encoded[y_train == 'decrease', 0] = 1        
        y_train_tensor = torch.tensor(labels_encoded, dtype=torch.float32).clone().detach()

        yield X_batch, y_train_tensor

    # Free up memory
    del train_dataset, train_loader
    del labels_encoded
    del y_train, X_batch, y_batch, y_train_tensor

def prepare_train_data(train_data = None, stock_len=2201, seq_length = 100, batch_size = 32):
  num_workers=multiprocessing.cpu_count() #parallel computing

  # Optimization 4 (Using data generator)
  generator = data_generator(train_data, stock_len, seq_length, batch_size, num_workers) 

  return generator

In [ ]:
num_layers_FC = ([1]*5) + ([2]*5)
num_layers_FC.reverse()
hidden_dim_LSTM = [i for i in range(21, 31)]
hidden_dim_LSTM.reverse()
params_dict = {"input_dim": [100]*10, 
               "hidden_dim_FC": [i for i in range(21, 31)],
               "output_dim": [3]*10, 
               "num_layers_LSTM":([1]*5) + ([2]*5), 
               "learning rate":[0.001, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09], 
               "num_epochs": [i for i in range(10, 21)],
               "model_id":[1, 2, 3, 4, 5, 6, 7, 8, 9, 10] }
params_dict["num_layers_FC"] = num_layers_FC
params_dict["hidden_dim_LSTM"] = hidden_dim_LSTM 

In [ ]:
#Necessary Imports
import torch
import torch.nn as nn
import torch.nn.init as init
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, precision_score
from tqdm import tqdm

In [ ]:
#num_layers = 1
#nn.dropout(p = 0.2)
for itr in range(0, 10):
  class LSTM1(nn.Module):
    def __init__(self, input_dim=params_dict["input_dim"][itr], 
                 hidden_dim_LSTM=params_dict["hidden_dim_LSTM"][itr],
                 hidden_dim_FC=params_dict["hidden_dim_FC"][itr],
                 output_dim=params_dict["output_dim"][itr],
                 num_LSTM_layers=params_dict["num_layers_LSTM"][itr],
                 num_layers_FC=params_dict["num_layers_FC"][itr]):
        
          super(LSTM1, self).__init__()
          self.lstm = nn.LSTM(input_dim, hidden_dim_LSTM, num_layers=num_LSTM_layers, batch_first=True) #num_layers = 2
          self.dropout = nn.Dropout(p=0.2)
          self.fc = nn.Linear(hidden_dim_LSTM, output_dim)
          self.softmax = nn.Softmax(dim=1)
          self.num_LSTM_layers = num_LSTM_layers
          self.hidden_dim_LSTM = hidden_dim_LSTM 
          
          # Use Xavier initialization for weights
          init.xavier_uniform_(self.lstm.weight_ih_l0)
          init.orthogonal_(self.lstm.weight_hh_l0)
          init.constant_(self.lstm.bias_ih_l0, 0.0)
          init.constant_(self.lstm.bias_hh_l0, 0.0)
          init.xavier_uniform_(self.fc.weight)
          init.constant_(self.fc.bias, 0.0)

    def forward(self, x):
        h0 = torch.zeros(self.num_LSTM_layers, x.size(0), self.hidden_dim_LSTM).requires_grad_().to(device)
        c0 = torch.zeros(self.num_LSTM_layers, x.size(0), self.hidden_dim_LSTM).requires_grad_().to(device)
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.dropout(out)
        out = self.fc(out[:, -1, :])
        out = out.view(-1, 3)
        out = self.softmax(out)
        return out
  # Check if CUDA is available
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#--------------------------------------------------------------------------------------------------------------------------------------------  
  def train_model1(batch_generator, model_id=params_dict["model_id"][itr], 
                input_dim=params_dict["input_dim"][itr], 
                learning_rate=params_dict["learning rate"][itr], 
                num_epochs=params_dict["num_epochs"][itr], 
                hidden_dim_LSTM=params_dict["hidden_dim_LSTM"][itr],
                output_dim=params_dict["output_dim"][itr]):
        # Evaluate each model for 10 epochs
        # Initialize the model
        learning_rate = learning_rate
        num_epochs = num_epochs
        input_dim = input_dim
        hidden_dim_LSTM = hidden_dim_LSTM
        # output_dim = y_train_tensor.shape[1]
        output_dim = 3

        # Define the loss function and optimizer
        criterion = nn.CrossEntropyLoss()  

        # Initialize the model
        print('Model no', model_id)
        model = LSTM1(input_dim, hidden_dim_LSTM, output_dim).to(device)

        # Define the optimizer and scheduler for the current model
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

        # Checkpointing parameters for the current model
        checkpoint_interval = 1
        checkpoint_file = f'lstm_task_3_checkpoint_{model_id}.pth'

        # Define early stopping parameters
        best_loss = float('inf')
        early_stop_counter = 0
        early_stop_patience = 5

        # Train the current model for 10 epochs
        for epoch in range(num_epochs):
            # Set model to train mode
            model.train()
            # model.double()

            # Iterate over batches
            for x_batch, y_batch in tqdm(generator):
                # Move data to device
                x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                # Cast x_batch to float32
                x_batch = x_batch.float()
                print(x_batch.shape)

                # Forward pass
                outputs = model(x_batch)
                loss = criterion(outputs, y_batch)

                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            # Print the loss for every 10 epochs
            if (epoch + 1) % 10 == 0:
                print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch + 1, num_epochs, loss.item()))

            # Check if current model has reached early stopping criteria
            print(loss.item())
            if loss.item() < best_loss:
                best_loss = loss.item()
                early_stop_counter = 0
            else:
                early_stop_counter += 1

            if early_stop_counter >= early_stop_patience:
                print(f"Training for model {itr} stopped early at epoch {epoch+1} due to early stopping")
                break

            # Save checkpoint at regular intervals
            if (epoch + 1) % checkpoint_interval == 0:
                torch.save({
                    'epoch': epoch + 1,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': loss.item(),
                }, checkpoint_file)

            # Step the scheduler
            scheduler.step()

        # Delete variables to free up memory
        del model, optimizer, scheduler
#------------------------------------------------------------------------------------------------------------------------------------------------------------
  # Empty the cache to free up GPU memory
  torch.cuda.empty_cache()
  seq_length = 100 # Define sequence length
  batch_size = 32 # Create data loader
  stocks_500 = 2201*500
  generator = prepare_train_data(train_data.iloc[:stocks_500, :], 2201, seq_length, batch_size)
#------------------------------------------------------------------------------------------------------------------------------------------------------------
  train_model1(generator, itr)

Model no 0


0it [00:06, ?it/s]


KeyboardInterrupt: ignored

In [ ]:
def print_table(metrics):
    header = ['Model', 'Train', '', '', 'Validation', '', '', '', '']
    subheader = ['', '', 'Precision', 'Accuracy', '% Positive', 'Precision', 'Accuracy', '% Positive']
    print(" {:<10} {:<10} {:<14} {:<10} {:<10} {:<14} {:<14} {:<14}".format(*header))
    print("{:<10} {:<0} {:<10} {:<10} {:<14} {:<10} {:<10} {:<14}".format(*subheader))

    for i in range(len(metrics)):
        row =[i+1] + [f"{val:.4f}" for val in metrics[i]['train']] + [f"{val:.4f}" for val in metrics[i]['val']]
        print("{:<11} {:<10} {:<10} {:<14} {:<10} {:<10} {:<14}".format(*row))
    return

In [ ]:
metrics = []
for itr2 in range(0, 10):
  class LSTM1(nn.Module):
    def __init__(self, input_dim=params_dict["input_dim"][itr2], 
                 hidden_dim_LSTM=params_dict["hidden_dim_LSTM"][itr2],
                 hidden_dim_FC=params_dict["hidden_dim_FC"][itr2],
                 output_dim=params_dict["output_dim"][itr2],
                 num_LSTM_layers=params_dict["num_layers_LSTM"][itr2],
                 num_layers_FC=params_dict["num_layers_FC"][itr2]):
        
          super(LSTM1, self).__init__()
          self.lstm = nn.LSTM(input_dim, hidden_dim_LSTM, num_layers=num_LSTM_layers, batch_first=True) #num_layers = 2
          self.dropout = nn.Dropout(p=0.2)
          self.fc = nn.Linear(hidden_dim_LSTM, output_dim)
          self.softmax = nn.Softmax(dim=1)
          self.num_LSTM_layers = num_LSTM_layers
          self.hidden_dim_LSTM = hidden_dim_LSTM 
          
          # Use Xavier initialization for weights
          init.xavier_uniform_(self.lstm.weight_ih_l0)
          init.orthogonal_(self.lstm.weight_hh_l0)
          init.constant_(self.lstm.bias_ih_l0, 0.0)
          init.constant_(self.lstm.bias_hh_l0, 0.0)
          init.xavier_uniform_(self.fc.weight)
          init.constant_(self.fc.bias, 0.0)

    def forward(self, x):
        h0 = torch.zeros(self.num_LSTM_layers, x.size(0), self.hidden_dim_LSTM).requires_grad_().to(device)
        c0 = torch.zeros(self.num_LSTM_layers, x.size(0), self.hidden_dim_LSTM).requires_grad_().to(device)
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.dropout(out)
        out = self.fc(out[:, -1, :])
        out = out.view(-1, 3)
        out = self.softmax(out)
        return out
  # Check if CUDA is available
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#------------------------------------------------------------------------------------------------------------------------------------------  
  def perform_validation1(num_models=1, file_desc=itr2, data_generator=None):
    # Evaluate each model on the primary training set and the validation set  
    checkpoint_file = f'lstm_task_3_checkpoint_{file_desc}.pth' # Load the trained model from the checkpoint file
    checkpoint = torch.load(checkpoint_file, map_location=torch.device('cpu'))  
    model = LSTM1(params_dict["input_dim"][itr2], params_dict["hidden_dim_LSTM"][itr2], 3).to(device)
    model.load_state_dict(checkpoint['model_state_dict'])

    # Evaluate the current model on the passed data  
    result_pred_labels = []  
    precisions = []
    accuracies = []
    for X_data, y_data in data_generator:        
      with torch.no_grad():
        X_data = X_data.float()
        outputs = model.forward(X_data.to(device))

      # Convert predicted and true labels to one-hot encodings
      predicted_labels = torch.argmax(outputs, dim=1).to(device)
      true_labels = torch.argmax(y_data, dim=1).to(device)

      # Calculate precision for each class    
      for i in range(3):
        if i == 2: # positive class
            true_positives = torch.sum((predicted_labels == i) & (true_labels == i))
            false_positives = torch.sum((predicted_labels == i) & (true_labels != 2))
            precision = true_positives.float() / (true_positives + false_positives).float()            
            accuracies.append(torch.mean((predicted_labels == true_labels).float() * (true_labels == i).float()))
        else: # negative class
            true_negatives = torch.sum((predicted_labels == i) & (true_labels == i) & (true_labels != 2))
            false_positives = torch.sum((predicted_labels == i) & (true_labels != i) & (true_labels != 2))
            precision = true_negatives.float() / (true_negatives + false_positives).float()
            accuracies.append(torch.mean((predicted_labels == true_labels).float() * (true_labels != i).float()))
        if not torch.isnan(precision):
          precisions.append(precision)

    # Calculate average precision and accuracy
    avg_precision = torch.mean(torch.tensor(precisions))
    avg_accuracy = torch.mean(torch.tensor(accuracies))

    # Calculate percentage of positive predictions
    percent_positive = torch.mean((predicted_labels == 2).float())
    
    # Free Memory  
    del model, outputs, X_data, y_data, true_labels, predicted_labels, precisions, accuracies, true_negatives, false_positives, precision
    del result_pred_labels
    torch.cuda.empty_cache()
    return [avg_precision, avg_accuracy, percent_positive]
#-----------------------------------------------------------------------------------------------------------------------------------------------  
  start = 0
  test_data = train_data.iloc[500*2201:599*2201, :]
  generator = prepare_train_data(test_data, 2201, seq_length=100, batch_size=1000)
#----------------------------------------------------------------------------------------------------------------------------------------------------
  metrics.append(perform_validation1(num_models=1, file_desc=itr2, data_generator=generator))


<ipython-input-4-db319c480757>:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train_tensor = torch.tensor(labels_encoded, dtype=torch.float32).clone().detach()


In [3]:
metrics

NameError: ignored

In [ ]:
def perform_validation1(num_models=1, file_desc=None, data_generator=None, param_dict=None):
  # Evaluate each model on the primary training set and the validation set  
  checkpoint_file = f'lstm_task_3_checkpoint_{file_desc}.pth' # Load the trained model from the checkpoint file
  checkpoint = torch.load(checkpoint_file, map_location=torch.device('cpu'))  
  model = LSTM1(param_dict["input_dim"], param_dict["hidden_dim_LSTM"], 3).to(device)
  model.load_state_dict(checkpoint['model_state_dict'])

  # Evaluate the current model on the passed data  
  result_pred_labels = []  
  precisions = []
  accuracies = []
  for X_data, y_data in data_generator:        
    with torch.no_grad():
      X_data = X_data.float()
      outputs = model.forward(X_data.to(device))

    # Convert predicted and true labels to one-hot encodings
    predicted_labels = torch.argmax(outputs, dim=1).to(device)
    true_labels = torch.argmax(y_data, dim=1).to(device)

    # Calculate precision for each class    
    for i in range(3):
      if i == 2: # positive class
          true_positives = torch.sum((predicted_labels == i) & (true_labels == i))
          false_positives = torch.sum((predicted_labels == i) & (true_labels != 2))
          precision = true_positives.float() / (true_positives + false_positives).float()
          accuracies.append(torch.mean((predicted_labels == true_labels).float() * (true_labels == i).float()))
      else: # negative class
          true_negatives = torch.sum((predicted_labels == i) & (true_labels == i) & (true_labels != 2))
          false_positives = torch.sum((predicted_labels == i) & (true_labels != i) & (true_labels != 2))
          precision = true_negatives.float() / (true_negatives + false_positives).float()
          accuracies.append(torch.mean((predicted_labels == true_labels).float() * (true_labels != i).float()))
      precisions.append(precision)

  # Calculate average precision and accuracy
  avg_precision = torch.mean(torch.tensor(precisions))
  avg_accuracy = torch.mean(torch.tensor(accuracies))

  # Calculate percentage of positive predictions
  percent_positive = torch.mean((predicted_labels == 2).float())
  
  # Free Memory  
  del model, outputs, X_data, y_data, true_labels, predicted_labels, precisions, accuracies, true_negatives, false_positives, precision
  del result_pred_labels
  torch.cuda.empty_cache()
  return [avg_precision, avg_accuracy, percent_positive]

In [ ]:
def print_table(metrics):
    header = ['Model', 'Train', '', '', 'Validation', '', '', '', '']
    subheader = ['', '', 'Precision', 'Accuracy', '% Positive', 'Precision', 'Accuracy', '% Positive']
    print(" {:<10} {:<10} {:<14} {:<10} {:<10} {:<14} {:<14} {:<14}".format(*header))
    print("{:<10} {:<0} {:<10} {:<10} {:<14} {:<10} {:<10} {:<14}".format(*subheader))

    for i in range(len(metrics)):
        row =[i+1] + [f"{val:.4f}" for val in metrics[i]['train']] + [f"{val:.4f}" for val in metrics[i]['val']]
        print("{:<11} {:<10} {:<10} {:<14} {:<10} {:<10} {:<14}".format(*row))
    return

In [ ]:
# Concatenate tensors of empty(Provding early shape can provide performance improvement)
start = 0
test_data = train_data.iloc[:100]
generator = prepare_train_data(test_data, 2201, seq_length, batch_size)

In [ ]:
# X_val_subsets, y_val_subsets = create_subset(X_test_tensor, y_test_tensor, num_models)
metrics = perform_validation1(num_models=1, file_desc=None, data_generator=None, param_dict=None)
print_table(metrics)

In [ ]:
metrics

[{'train': [], 'val': [tensor(nan), tensor(0.1481), tensor(0.2778)]}]